In [2]:
from pulp import LpProblem, LpVariable, LpMaximize, LpMinimize, LpStatus, lpSum, value, LpConstraint
from pulp import PULP_CBC_CMD
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import seaborn as sns
from  itertools import product
import math
import time
import os

In [3]:
# Read files
files = glob('../*/*routing_ver2/*.csv')
# print(files)
dfs, name_df = [], []
for i, file in enumerate(files):
    df_name = os.path.splitext(os.path.basename(files[i]))[0]
    name_df.append(df_name)
    df = pd.read_csv(files[i])
    dfs.append(df)
print(name_df)

['distances', 'locations', 'orders', 'small_distances', 'small_locations', 'small_orders']


In [4]:
for df in dfs[3:]:
    print(df.shape)
    display(df.head(3))

(25, 3)


,k1,k2,t
0,p,p,0.000000
1,p,s1,2.000000
2,p,s2,2.828427


(5, 4)


,k,x,y,depo_flag
0,p,0,0,1
1,s1,-2,0,0
2,s2,-2,-2,0


(8, 5)


,r,s,w,b,e
0,r1,s1,1000,1,1
1,r2,s2,1000,1,2
2,r3,s3,1000,2,2


In [6]:
kk_df = dfs[3].copy()
k_df = dfs[4].copy()
r_df = dfs[5].copy()

In [25]:
# 日付
D = list(range(r_df['b'].min(), r_df['e'].max()+1))
D
# ベース拠点
P = k_df['k'][k_df['depo_flag'] == 1]
# お店
S = k_df['k'][k_df['depo_flag'] == 0]
P, S
# すべての拠点
K = k_df['k']
# 荷物
R = r_df['r']
K, R

# 日付・拠点1・拠点2
DKK = [(d, k1, k2) for d in D for k1 in K for k2 in K]
# 日付・拠点
DK = [(d, k) for d in D for k in K]
# 日付・荷物
DR = [(d, r) for d in D for r in R]
# 荷物・お店
R2S = {r.r: r.s for r in r_df.itertuples()}
# 荷物・重量
R2W = {r.r: r.w for r in r_df.itertuples()}
# 荷物・配送日1
R2B = {r.r: r.b for r in r_df.itertuples()}
# 荷物・配送日2
R2E = {r.r: r.e for r in r_df.itertuples()}
# 拠点1・拠点2・時間
KK2T = {(r.k1, r.k2): r.t for r in kk_df.itertuples()}

In [31]:
x = LpVariable.dicts('x', DKK, cat='Binary')
u = LpVariable.dicts('u', DK)
y = LpVariable.dicts('y', DR, cat='Binary')
h = LpVariable.dicts('h', D, cat='Continuous', lowBound=0)

In [35]:
m = LpProblem('vrp_small', sense=LpMinimize)

OVERWORK_HOURLY_RATE = 3000
WEIGHT_UNIT_PRICE = 46
overwork_cost = lpSum([OVERWORK_HOURLY_RATE * h[d] for d in D])
outsource_cost = lpSum(WEIGHT_UNIT_PRICE * R2W[r] * (1-lpSum([y[d, r] for d in D])) for r in R)
m += overwork_cost + outsource_cost



m

vrp_small:
MINIMIZE
3000*h_1 + 3000*h_2 + 3000*h_3 + 3000*h_4 + -46000*y_(1,_'r1') + -46000*y_(1,_'r2') + -46000*y_(1,_'r3') + -69000*y_(1,_'r4') + -69000*y_(1,_'r5') + -69000*y_(1,_'r6') + -69000*y_(1,_'r7') + -138000*y_(1,_'r8') + -46000*y_(2,_'r1') + -46000*y_(2,_'r2') + -46000*y_(2,_'r3') + -69000*y_(2,_'r4') + -69000*y_(2,_'r5') + -69000*y_(2,_'r6') + -69000*y_(2,_'r7') + -138000*y_(2,_'r8') + -46000*y_(3,_'r1') + -46000*y_(3,_'r2') + -46000*y_(3,_'r3') + -69000*y_(3,_'r4') + -69000*y_(3,_'r5') + -69000*y_(3,_'r6') + -69000*y_(3,_'r7') + -138000*y_(3,_'r8') + -46000*y_(4,_'r1') + -46000*y_(4,_'r2') + -46000*y_(4,_'r3') + -69000*y_(4,_'r4') + -69000*y_(4,_'r5') + -69000*y_(4,_'r6') + -69000*y_(4,_'r7') + -138000*y_(4,_'r8') + 552000
VARIABLES
h_1 Continuous
h_2 Continuous
h_3 Continuous
h_4 Continuous
0 <= y_(1,_'r1') <= 1 Integer
0 <= y_(1,_'r2') <= 1 Integer
0 <= y_(1,_'r3') <= 1 Integer
0 <= y_(1,_'r4') <= 1 Integer
0 <= y_(1,_'r5') <= 1 Integer
0 <= y_(1,_'r6') <= 1 Integer
0 <